This block imports the necessary Python libraries for the task:

- **os**: For interacting with the operating system.
- **torch**: PyTorch library for deep learning.
- **nn**: Neural network modules and loss functions.
- **DataLoader, Dataset**: For handling datasets and creating data loaders.
- **BertTokenizer, BertModel, AdamW, get_linear_schedule_with_warmup**: Transformers library for BERT.
- **train_test_split, accuracy_score, classification_report**: For evaluating model performance.
- **pandas**: For data manipulation and analysis.


In [44]:
# Importing necessary libraries
import os  # For interacting with the operating system
import torch  # PyTorch library for deep learning
from torch import nn  # Neural network modules and loss functions
from torch.utils.data import DataLoader, Dataset  # For handling datasets and creating data loaders
from transformers import BertTokenizer, BertModel, AdamW, get_linear_schedule_with_warmup  # Transformers library for BERT
from sklearn.model_selection import train_test_split  # For splitting data into training and validation sets
from sklearn.metrics import accuracy_score, classification_report  # For evaluating model performance
import pandas as pd  # For data manipulation and analysis


In [45]:
# Loading data
def load_data(data_file):
    """
    Function to load data from a CSV file.
    
    Args:
    data_file (str): Path to the CSV file containing the data.
    
    Returns:
    texts (list): List of texts.
    labels (list): List of corresponding labels.
    """
    df = pd.read_csv(data_file)  # Read data from the CSV file into a DataFrame
    texts = df['text'].tolist()  # Extract texts column and convert to list
    labels = df['author'].tolist()  # Extract labels column and convert to list
    return texts, labels

data_file = "/kaggle/input/persian-authors-preprocessed/persian_authors_preprocessed.csv"  # Provide the path to your dataset file
texts, labels = load_data(data_file)  # Load data from the CSV file


In [46]:
unique_labels = sorted(list(set(labels)))  # Get unique labels and sort them
print(unique_labels)  # Print unique labels

dict_label = {}  # Initialize an empty dictionary for label mapping
for l in range(len(unique_labels)):
    dict_label[int(l)] = unique_labels[l]  # Map integer labels to original labels
    dict_label[unique_labels[l]] = int(l)  # Map original labels to integer labels
    print(unique_labels[l], str(l))  # Print label and corresponding integer label


['eraghi', 'ferdousi', 'jami', 'jooya', 'moulavi', 'nezami', 'rahi', 'saadi', 'saeb', 'shahriar']
eraghi 0
ferdousi 1
jami 2
jooya 3
moulavi 4
nezami 5
rahi 6
saadi 7
saeb 8
shahriar 9


In [47]:
class TextClassificationDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length, dict_label={}, overwrite_dict=True):
        """
        Custom dataset class for text classification.
        
        Args:
        texts (list): List of texts.
        labels (list): List of corresponding labels.
        tokenizer: Tokenizer object from the transformers library.
        max_length (int): Maximum length of the input sequences.
        dict_label (dict): Dictionary mapping labels to integers and vice versa.
        overwrite_dict (bool): Whether to overwrite the label dictionary if provided.
        """
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.unique_labels = sorted(list(set(labels)))
        self.unique_labels = list(self.unique_labels)
        self.dict_label = dict_label
        if overwrite_dict:
            self.dict_label = {}
            for l in range(len(self.unique_labels)):
                self.dict_label[int(l)] = self.unique_labels[l]
                self.dict_label[self.unique_labels[l]] = int(l)
            # Check unique labels
            unique_labels = set(labels)
            num_classes = len(unique_labels)
            print("Unique labels:", unique_labels)
            print("Number of classes:", num_classes)
            label_counts = {label: labels.count(label) for label in unique_labels}
            print("Label counts:", label_counts)
            print(self.dict_label)

    def get_dict(self):
        """
        Get the label dictionary.
        
        Returns:
        dict_label (dict): Dictionary mapping labels to integers and vice versa.
        """
        return self.dict_label

    def __len__(self):
        """
        Get the total number of samples in the dataset.
        
        Returns:
        length (int): Total number of samples.
        """
        return len(self.texts)

    def __getitem__(self, idx):
        """
        Get a single sample from the dataset.
        
        Args:
        idx (int): Index of the sample to retrieve.
        
        Returns:
        sample (dict): Dictionary containing input_ids, attention_mask, and label tensors.
        """
        text = self.texts[idx]
        label = self.dict_label[self.labels[idx]]  # Convert label to integer
        encoding = self.tokenizer(text, return_tensors='pt', max_length=self.max_length, padding='max_length', truncation=True)
        return {'input_ids': encoding['input_ids'].flatten(), 'attention_mask': encoding['attention_mask'].flatten(), 'label': torch.tensor(label)}


In [48]:
# Defining the BERT classifier model
class BERTClassifier(nn.Module):
    def __init__(self, bert_model_name, num_classes):
        """
        BERT-based classifier model.
        
        Args:
        bert_model_name (str): Name of the pre-trained BERT model.
        num_classes (int): Number of classes for classification.
        """
        super(BERTClassifier, self).__init__()
        self.bert = BertModel.from_pretrained(bert_model_name)  # Load pre-trained BERT model
        self.dropout = nn.Dropout(0.1)  # Dropout layer for regularization
        self.fc = nn.Linear(self.bert.config.hidden_size, num_classes)  # Fully connected layer for classification

    def forward(self, input_ids, attention_mask):
        """
        Forward pass of the model.
        
        Args:
        input_ids (torch.Tensor): Tensor of input token IDs.
        attention_mask (torch.Tensor): Tensor of attention masks.
        
        Returns:
        logits (torch.Tensor): Output logits from the classifier.
        """
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)  # BERT forward pass
        pooled_output = outputs.pooler_output  # Extract pooled output
        x = self.dropout(pooled_output)  # Apply dropout
        logits = self.fc(x)  # Pass through fully connected layer
        return logits  # Return logits


In [49]:
def train(model, data_loader, optimizer, scheduler, device):
    """
    Function to train the model.
    
    Args:
    model (nn.Module): The neural network model to be trained.
    data_loader (DataLoader): DataLoader object containing the training data.
    optimizer: Optimizer object for updating model parameters.
    scheduler: Learning rate scheduler.
    device (torch.device): Device to run the training on.
    """
    model.train()  # Set the model to train mode
    for batch in data_loader:
        optimizer.zero_grad()  # Clear gradients
        input_ids = batch['input_ids'].to(device)  # Move input_ids to device
        attention_mask = batch['attention_mask'].to(device)  # Move attention_mask to device
        labels = batch['label'].to(device)  # Move labels to device
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)  # Forward pass
        loss = nn.CrossEntropyLoss()(outputs, labels)  # Calculate loss
        loss.backward()  # Backpropagation
        optimizer.step()  # Update model parameters
        scheduler.step()  # Update learning rate


In [50]:
# Evaluation function
def evaluate(model, data_loader, device):
    """
    Function to evaluate the model.
    
    Args:
    model (nn.Module): The neural network model to be evaluated.
    data_loader (DataLoader): DataLoader object containing the evaluation data.
    device (torch.device): Device to run the evaluation on.
    
    Returns:
    accuracy (float): Accuracy score.
    classification_report (str): Classification report containing precision, recall, F1-score, and support.
    """
    model.eval()  # Set the model to evaluation mode
    predictions = []  # List to store predicted labels
    actual_labels = []  # List to store actual labels
    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)  # Move input_ids to device
            attention_mask = batch['attention_mask'].to(device)  # Move attention_mask to device
            labels = batch['label'].to(device)  # Move labels to device
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)  # Forward pass
            _, preds = torch.max(outputs, dim=1)  # Get predicted labels
            predictions.extend(preds.cpu().tolist())  # Append predicted labels to the list
            actual_labels.extend(labels.cpu().tolist())  # Append actual labels to the list
    # Calculate accuracy and classification report
    accuracy = accuracy_score(actual_labels, predictions)
    classification_report_str = classification_report(actual_labels, predictions)
    return accuracy, classification_report_str  # Return accuracy and classification report


In [51]:
# Function for predicting sentiment
def predict_sentiment(text, model, tokenizer, device, max_length=128, dict_label={}):
    """
    Function to predict sentiment of a given text.
    
    Args:
    text (str): The input text to predict sentiment for.
    model (nn.Module): The trained neural network model.
    tokenizer: Tokenizer object from the transformers library.
    device (torch.device): Device to run the prediction on.
    max_length (int): Maximum length of the input sequence.
    dict_label (dict): Dictionary mapping labels to integers and vice versa.
    
    Returns:
    sentiment (str): Predicted sentiment label.
    """
    model.eval()  # Set the model to evaluation mode
    encoding = tokenizer(text, return_tensors='pt', max_length=max_length, padding='max_length', truncation=True)  # Tokenize input text
    input_ids = encoding['input_ids'].to(device)  # Move input_ids to device
    attention_mask = encoding['attention_mask'].to(device)  # Move attention_mask to device

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)  # Forward pass
        _, preds = torch.max(outputs, dim=1)  # Get predicted label
    return dict_label[preds.item()]  # Return predicted sentiment label


In [52]:
# Set up parameters
bert_model_name = 'bert-base-uncased'
num_classes = len(set(labels))
print("num_classes", num_classes)
max_length = 512
batch_size = 16
num_epochs = 60
learning_rate = 2e-5


num_classes 10


In [53]:
# Splitting data into train and validation sets
train_texts, val_texts, train_labels, val_labels = train_test_split(texts, labels, test_size=0.2, random_state=42)


In [54]:
# Initializing tokenizer and datasets
tokenizer = BertTokenizer.from_pretrained(bert_model_name)  # Initialize BERT tokenizer
train_dataset = TextClassificationDataset(train_texts, train_labels, tokenizer, max_length)  # Create training dataset
dict_label = train_dataset.get_dict()  # Get label dictionary
val_dataset = TextClassificationDataset(val_texts, val_labels, tokenizer, max_length, dict_label, overwrite_dict=False)  # Create validation dataset
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)  # Create training dataloader
val_dataloader = DataLoader(val_dataset, batch_size=batch_size)  # Create validation dataloader


Unique labels: {'rahi', 'ferdousi', 'jooya', 'jami', 'moulavi', 'eraghi', 'saadi', 'nezami', 'shahriar', 'saeb'}
Number of classes: 10
Unique labels: {'rahi', 'ferdousi', 'jooya', 'jami', 'moulavi', 'eraghi', 'saadi', 'nezami', 'shahriar', 'saeb'}
Number of classes: 10
Unique labels: {'rahi', 'ferdousi', 'jooya', 'jami', 'moulavi', 'eraghi', 'saadi', 'nezami', 'shahriar', 'saeb'}
Number of classes: 10
Unique labels: {'rahi', 'ferdousi', 'jooya', 'jami', 'moulavi', 'eraghi', 'saadi', 'nezami', 'shahriar', 'saeb'}
Number of classes: 10
Unique labels: {'rahi', 'ferdousi', 'jooya', 'jami', 'moulavi', 'eraghi', 'saadi', 'nezami', 'shahriar', 'saeb'}
Number of classes: 10
Unique labels: {'rahi', 'ferdousi', 'jooya', 'jami', 'moulavi', 'eraghi', 'saadi', 'nezami', 'shahriar', 'saeb'}
Number of classes: 10
Unique labels: {'rahi', 'ferdousi', 'jooya', 'jami', 'moulavi', 'eraghi', 'saadi', 'nezami', 'shahriar', 'saeb'}
Number of classes: 10
Unique labels: {'rahi', 'ferdousi', 'jooya', 'jami', 'm

In [55]:
# Setting up device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # Check if GPU is available
model = BERTClassifier(bert_model_name, num_classes).to(device)  # Move model to device


In [56]:
# Initializing optimizer and scheduler
optimizer = AdamW(model.parameters(), lr=learning_rate, no_deprecation_warning=True)  # Initialize AdamW optimizer
total_steps = len(train_dataloader) * num_epochs  # Calculate total training steps
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)  # Initialize scheduler


In [57]:
# Training the model
for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs}")
    train(model, train_dataloader, optimizer, scheduler, device)  # Train the model
    accuracy, report = evaluate(model, val_dataloader, device)  # Evaluate the model on validation data
    print(f"Validation Accuracy: {accuracy:.4f}")  # Print validation accuracy
    print(report)  # Print classification report


Epoch 1/100


/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Validation Accuracy: 0.0484
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         7
           1       0.00      0.00      0.00         8
           2       0.00      0.00      0.00         9
           3       0.05      1.00      0.09         3
           4       0.00      0.00      0.00         5
           5       0.00      0.00      0.00         3
           6       0.00      0.00      0.00         4
           7       0.00      0.00      0.00         9
           8       0.00      0.00      0.00         7
           9       0.00      0.00      0.00         7

    accuracy                           0.05        62
   macro avg       0.00      0.10      0.01        62
weighted avg       0.00      0.05      0.00        62

Epoch 2/100


/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Validation Accuracy: 0.0484
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         7
           1       0.00      0.00      0.00         8
           2       0.00      0.00      0.00         9
           3       0.05      1.00      0.09         3
           4       0.00      0.00      0.00         5
           5       0.00      0.00      0.00         3
           6       0.00      0.00      0.00         4
           7       0.00      0.00      0.00         9
           8       0.00      0.00      0.00         7
           9       0.00      0.00      0.00         7

    accuracy                           0.05        62
   macro avg       0.00      0.10      0.01        62
weighted avg       0.00      0.05      0.00        62

Epoch 3/100


...

Validation Accuracy: 0.5323
              precision    recall  f1-score   support

           0       0.60      0.43      0.50         7
           1       0.73      1.00      0.84         8
           2       0.50      0.33      0.40         9
           3       0.30      1.00      0.46         3
           4       1.00      0.40      0.57         5
           5       0.00      0.00      0.00         3
           6       0.50      1.00      0.67         4
           7       0.55      0.67      0.60         9
           8       0.50      0.43      0.46         7
           9       0.33      0.14      0.20         7

    accuracy                           0.53        62
   macro avg       0.50      0.54      0.47        62
weighted avg       0.53      0.53      0.50        62

Epoch 13/100


/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Validation Accuracy: 0.4355
              precision    recall  f1-score   support

           0       0.75      0.43      0.55         7
           1       1.00      0.62      0.77         8
           2       0.00      0.00      0.00         9
           3       0.18      1.00      0.30         3
           4       0.40      0.80      0.53         5
           5       0.00      0.00      0.00         3
           6       0.57      1.00      0.73         4
           7       0.64      0.78      0.70         9
           8       1.00      0.14      0.25         7
           9       0.00      0.00      0.00         7

    accuracy                           0.44        62
   macro avg       0.45      0.48      0.38        62
weighted avg       0.50      0.44      0.40        62

Epoch 14/100


/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Validation Accuracy: 0.5161
              precision    recall  f1-score   support

           0       1.00      0.29      0.44         7
           1       0.80      1.00      0.89         8
           2       1.00      0.22      0.36         9
           3       0.30      1.00      0.46         3
           4       0.67      0.40      0.50         5
           5       0.00      0.00      0.00         3
           6       0.50      1.00      0.67         4
           7       0.50      0.67      0.57         9
           8       0.40      0.57      0.47         7
           9       0.20      0.14      0.17         7

    accuracy                           0.52        62
   macro avg       0.54      0.53      0.45        62
weighted avg       0.60      0.52      0.48        62

Epoch 15/100
Validation Accuracy: 0.4516
              precision    recall  f1-score   support

           0       0.62      0.71      0.67         7
           1       0.57      1.00      0.73         8
         

In [58]:
# Saving the trained model
torch.save(model.state_dict(), "bert_classifier.pth")  # Save model state dictionary to file


In [60]:
# Testing sentiment prediction
accuracy = 0  # Initialize accuracy counter
for i in range(len(val_texts)):
    test_text = val_texts[i]  # Get test text
    sentiment = predict_sentiment(test_text, model, tokenizer, device, dict_label=dict_label)  # Predict sentiment
    if sentiment == val_labels[i]:
        accuracy += 1  # Increment accuracy if prediction matches actual label
    print(val_labels[i])  # Print actual sentiment label
    print(f"Predicted sentiment: {sentiment}")  # Print predicted sentiment
print(f"accuracy: {accuracy/len(val_texts)}")  # Print accuracy


eraghi
Predicted sentiment: eraghi
rahi
Predicted sentiment: eraghi
nezami
Predicted sentiment: ferdousi
jami
Predicted sentiment: jami
saadi
Predicted sentiment: saadi
shahriar
Predicted sentiment: nezami
ferdousi
Predicted sentiment: ferdousi
jami
Predicted sentiment: jami
eraghi
Predicted sentiment: eraghi
jami
Predicted sentiment: shahriar
eraghi
Predicted sentiment: rahi
jami
Predicted sentiment: moulavi
saeb
Predicted sentiment: jami
shahriar
Predicted sentiment: ferdousi
saadi
Predicted sentiment: jami
moulavi
Predicted sentiment: rahi
ferdousi
Predicted sentiment: ferdousi
moulavi
Predicted sentiment: moulavi
ferdousi
Predicted sentiment: ferdousi
saadi
Predicted sentiment: ferdousi
moulavi
Predicted sentiment: rahi
moulavi
Predicted sentiment: moulavi
saeb
Predicted sentiment: nezami
jooya
Predicted sentiment: jami
rahi
Predicted sentiment: saeb
saeb
Predicted sentiment: saeb
saadi
Predicted sentiment: jami
eraghi
Predicted sentiment: jami
jooya
Predicted sentiment: saeb
eragh